In [1]:
import taskingai
# Load TaskingAI API Key from environment variable

# TaskingAI Model Inference: Chat Completion

In [2]:
from taskingai.inference import *
import json
# choose an available chat_completion model from your project
model_id = "Gk1145Bl"

In [3]:
# normal 
chat_completion_result = taskingai.inference.chat_completion(
    model_id=model_id,
    messages=[
        SystemMessage("You are a professional assistant."),
        UserMessage("Hi"),
    ]
)
chat_completion_result

ChatCompletion(object='ChatCompletion', finish_reason=<ChatCompletionFinishReason.stop: 'stop'>, message=ChatCompletionAssistantMessage(content='Hello! How can I assist you today?', role=<ChatCompletionRole.assistant: 'assistant'>, function_call=None), created_timestamp=1701342218014)

In [4]:
# multi round chat completion
chat_completion_result = taskingai.inference.chat_completion(
    model_id=model_id,
    messages=[
        SystemMessage("You are a professional assistant."),
        UserMessage("Hi"),
        AssistantMessage("Hello! How can I assist you today?"),
        UserMessage("Can you tell me a joke?"),
        AssistantMessage("Sure, here is a joke for you: Why don't scientists trust atoms? Because they make up everything!"),
        UserMessage("That's funny. Can you tell me another one?"),
    ]
)
chat_completion_result

ChatCompletion(object='ChatCompletion', finish_reason=<ChatCompletionFinishReason.stop: 'stop'>, message=ChatCompletionAssistantMessage(content="Of course! How about this one: Why don't skeletons fight each other? They don't have the guts!", role=<ChatCompletionRole.assistant: 'assistant'>, function_call=None), created_timestamp=1701342220370)

In [5]:
# config max tokens
chat_completion_result = taskingai.inference.chat_completion(
    model_id=model_id,
    messages=[
        SystemMessage("You are a professional assistant."),
        UserMessage("Hi"),
        AssistantMessage("Hello! How can I assist you today?"),
        UserMessage("Can you tell me a joke?"),
        AssistantMessage("Sure, here is a joke for you: Why don't scientists trust atoms? Because they make up everything!"),
        UserMessage("That's funny. Can you tell me another one?"),
    ],
    configs={
        "max_tokens": 5
    }
)
chat_completion_result

ChatCompletion(object='ChatCompletion', finish_reason=<ChatCompletionFinishReason.length: 'length'>, message=ChatCompletionAssistantMessage(content="Of course! Here's", role=<ChatCompletionRole.assistant: 'assistant'>, function_call=None), created_timestamp=1701342221823)

In [6]:
# function call
function = Function(
    name="plus_a_and_b",
    description="Sum up a and b and return the result",
    parameters={
        "type": "object",
        "properties": {
            "a": {
                "type": "integer",
                "description": "The first number"
            },
            "b": {
                "type": "integer",
                "description": "The second number"
            }
        },
        "required": ["a", "b"]
    },
)
chat_completion_result = taskingai.inference.chat_completion(
    model_id=model_id,
    messages=[
        SystemMessage("You are a professional assistant."),
        UserMessage("What is the result of 112 plus 22?"),
    ],
    functions=[function]
)
print(f"chat_completion_result = {chat_completion_result}")

assistant_function_call_message = chat_completion_result.message
fucntion_name = assistant_function_call_message.function_call.name
argument_content = json.dumps(assistant_function_call_message.function_call.arguments)
print(f"function name: {fucntion_name}, argument content: {argument_content}")

chat_completion_result = object='ChatCompletion' finish_reason=<ChatCompletionFinishReason.function_calls: 'function_call'> message=ChatCompletionAssistantMessage(content=None, role=<ChatCompletionRole.assistant: 'assistant'>, function_call=ChatCompletionFunctionCall(arguments={'a': 112, 'b': 22}, name='plus_a_and_b')) created_timestamp=1701342223979
function name: plus_a_and_b, argument content: {"a": 112, "b": 22}


In [7]:
# add function message
chat_completion_result = taskingai.inference.chat_completion(
    model_id=model_id,
    messages=[
        SystemMessage("You are a professional assistant."),
        UserMessage("What is the result of 112 plus 22?"),
        assistant_function_call_message,
        FunctionMessage(name=fucntion_name, content="144")
    ],
    functions=[function]
)
chat_completion_result

ChatCompletion(object='ChatCompletion', finish_reason=<ChatCompletionFinishReason.stop: 'stop'>, message=ChatCompletionAssistantMessage(content='The result of 112 plus 22 is 134.', role=<ChatCompletionRole.assistant: 'assistant'>, function_call=None), created_timestamp=1701342226882)

In [14]:
# generate in stream mode
chat_completion_result = taskingai.inference.chat_completion(
    model_id=model_id,
    messages=[
        UserMessage("Please count from 1 to 100, and add a new line after every 10 numbers."),
    ],
    stream=True
)
for item in chat_completion_result:
    
    if item.get("object") == "ChatCompletionChunk":
        chunk = ChatCompletionChunk(**item)
        print(chunk.delta, end="", flush=True)
        
    elif item.get("object") == "ChatCompletion":
        result = ChatCompletion(**item)
        print(f"\n message object: {result.message}")

1 2 3 4 5 6 7 8 9 10 
11 12 13 14 15 16 17 18 19 20 
21 22 23 24 25 26 27 28 29 30 
31 32 33 34 35 36 37 38 39 40 
41 42 43 44 45 46 47 48 49 50 
51 52 53 54 55 56 57 58 59 60 
61 62 63 64 65 66 67 68 69 70 
71 72 73 74 75 76 77 78 79 80 
81 82 83 84 85 86 87 88 89 90 
91 92 93 94 95 96 97 98 99 100
 message object: content='1 2 3 4 5 6 7 8 9 10 \n11 12 13 14 15 16 17 18 19 20 \n21 22 23 24 25 26 27 28 29 30 \n31 32 33 34 35 36 37 38 39 40 \n41 42 43 44 45 46 47 48 49 50 \n51 52 53 54 55 56 57 58 59 60 \n61 62 63 64 65 66 67 68 69 70 \n71 72 73 74 75 76 77 78 79 80 \n81 82 83 84 85 86 87 88 89 90 \n91 92 93 94 95 96 97 98 99 100' role=<ChatCompletionRole.assistant: 'assistant'> function_call=None
